In [120]:
%load_ext autoreload
%autoreload 2

import os 
import sys 
import json

home = os.path.abspath(os.path.join(os.getcwd(), '..','..'))
pths = [os.path.join(home, 'astrobot')]

for pth in pths: 
    if pth not in sys.path: sys.path.insert(0, pth)
        
# # Import modules
# from pml.utils import *
# from pml.matrix_utils2 import MatrixUtils2 as mu
# #from pml.matrix_surface2 import MatrixSurface2
# from pml.b2 import B
# from pml.shapedataframe import ShapeDataFrame

import ladybug_geometry.geometry3d as geom3
import ladybug_geometry.geometry2d as geom2
# from ladybug_geometry.geometry2d.pointvector import Point2D
# from ladybug_geometry.geometry3d.face import Face3D
# from ladybug_geometry.geometry2d.polygon import Polygon2D
# from ladybug_geometry.geometry2d.line import LineSegment2D
from ladybug_geometry_polyskel import polyskel

import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as stats
import itertools

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [89]:
simdir_path = os.path.join('simulations', 'sim_0')

In [105]:
labels = ['wwr', 'theta']
# just uniform sampling for now
param_mtx = [np.linspace(0.2, 0.9, 5), 
             np.linspace(0, np.pi * 2, 5)]
param_mtx = np.array(list(itertools.product(*param_mtx))).T

# Save
dfparam = pd.DataFrame({label: row for row, label in zip(param_mtx, labels)})
dfparam.to_pickle(os.path.join(simdir_path, 'sample_mtx.pkl'))

In [129]:
dfparam = pd.read_pickle(os.path.join(simdir_path, 'sample_mtx.pkl'))

def compute_flr_verts(theta, area=72, dim=12, aspect_ratio=0.5):
    """Compute floor vertices from theta"""
    origin = geom3.pointvector.Point3D(0, 0, 0)
    ns, ew = aspect_ratio * dim, dim
    flr_verts = np.array([
        [-ew / 2., -ns / 2.,  0],
        [ ew / 2., -ns / 2.,  0],
        [ ew / 2.,  ns / 2.,  0],
        [-ew / 2.,  ns / 2.,  0]])
    pts = [geom3.pointvector.Point3D(*flr_vert) for flr_vert in flr_verts]
    normal = geom3.pointvector.Vector3D(0, 0, 1)
    plane = geom3.plane.Plane(normal, origin.duplicate())
    flr = geom3.face.Face3D(pts, plane).rotate(normal, theta, origin)
    scale_factor = np.sqrt(area / flr.area)
    if np.abs(scale_factor - 1.0) > 1e-10:
        flr = flr.scale(scale_factor, origin)
    return flr

# Generate the floors
dfparam['flr_verts'] = [compute_flr_verts(theta) for theta in dfparam['theta'].values]

In [ ]:
# Visualize